In [ ]:
import numpy as np
import pandas as pd
import pywt
import matplotlib.pyplot as plt
import os
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
from itertools import combinations
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import torch as th
import torch.nn as nn
import gym
from sklearn.decomposition import PCA
import torch.nn.functional as F
from torch.nn import MultiheadAttention
from scipy import stats
import seaborn as sns
import warnings
import random
warnings.filterwarnings('ignore')

# ==============================================================
# SEMILLAS ALEATORIAS MEJORADAS
# ==============================================================
def set_random_seeds(seed):
    """Set random seeds for reproducibility"""
    np.random.seed(seed)
    th.manual_seed(seed)
    th.cuda.manual_seed_all(seed)
    random.seed(seed)
    if hasattr(th.backends, 'cudnn'):
        th.backends.cudnn.deterministic = True
        th.backends.cudnn.benchmark = False

# ==============================================================
# CONFIGURATION CON TIMESTEPS REALISTAS Y MÉTRICAS COMPLETAS + MÉTODOS CLÁSICOS
# ==============================================================
POINT_VALUE = 50
COMMISSION_PER_TRADE = 0.25
SPREAD = 0.25

# 🚀 WAVELETS EXPANDIDAS (para responder a Reviewer #2)
wavelet_families = {
    'Daubechies': ['db1', 'db2', 'db4'],      
    'Symlets': ['sym1', 'sym4'],        
    'Coiflets': ['coif1', 'coif4'],
    'Biorthogonal': ['bior1.1','bior2.2']
}

# 🚀 HYPERPARAMETERS REALISTAS - CORREGIDO CON MÁS VARIACIÓN
HYPERPARAM_GRID = {
    'PPO': {
        'learning_rate': [3e-4, 1e-4],     
        'gamma': [0.95, 0.99],             
        'n_steps': [2048],                 
        'batch_size': [128]                
    },
    'A2C': {
        'learning_rate': [1e-4, 5e-5, 3e-4],    # MÁS VARIACIÓN
        'gamma': [0.95, 0.99],             
        'n_steps': [20, 50]                      # MÁS VARIACIÓN
    },
    'DQN': {
        'learning_rate': [1e-4, 5e-5, 3e-4],    # MÁS VARIACIÓN
        'gamma': [0.95, 0.99],             
        'buffer_size': [100000, 50000],          # MÁS VARIACIÓN
        'exploration_fraction': [0.1, 0.2]      # MÁS VARIACIÓN
    }
}

# 🚀 TIMESTEPS REALISTAS
TRAINING_TIMESTEPS = {
    'hyperparameter_tuning': 10000,    
    'final_training': 50000,           
    'validation_steps': 5000           
}

# 🚀 TODAS LAS COMBINACIONES DE FEATURES
MAIN_FEATURE_COMBINATIONS = [
    ('DIX',), ('GEX',), ('VIX',),
    ('DIX', 'GEX'), ('DIX', 'VIX'), ('GEX', 'VIX'),
    ('DIX', 'GEX', 'VIX')
]

# Paths
file_path = r"C:\Users\elect\OneDrive\Documentos\Doctorado\Articulo 4 peer review\Data\merged_data_with_vix_2.csv"
results_dir = r"C:\Users\elect\OneDrive\Documentos\Doctorado\Articulo 4 peer review\results_COMPLETE_WITH_CLASSICAL_METHODS"

# Create directories
directories = ['graphs', 'csv', 'out_of_sample', 'academic_reports', 'denoising_analysis', 'wavelet_analysis']
for dir_name in directories:
    os.makedirs(os.path.join(results_dir, dir_name), exist_ok=True)

print("🚀 COMPLETE CONFIGURATION WITH ALL IMPROVEMENTS FOR REVIEWERS:")
print(f"📊 Wavelet families: {len(wavelet_families)} ({sum(len(w) for w in wavelet_families.values())} wavelets)")
print(f"🤖 RL Algorithms: PPO, A2C, DQN")  
print(f"🔧 Classical Methods: XGBoost, Random Forest, Logistic Regression, Buy&Hold, Moving Average")
print(f"🔧 Feature combinations: {len(MAIN_FEATURE_COMBINATIONS)}")
print(f"🧠 Feature Extractor: ADVANCED (Attention + BiLSTM + Positional Encoding)")
print(f"📈 Strategy: LONG-ONLY (Buy/Hold - No Short Selling)")
print(f"🎯 Training timesteps: {TRAINING_TIMESTEPS['final_training']:,}")
print(f"📊 Metrics: Alpha, Beta, Sharpe, Sortino, Max DD, Win Rate, Max Loss, RoMaD, CAGR, Volatility, T-Stat, P-Value")
print(f"🔬 Analysis: Denoising effectiveness, Wavelet level justification, Statistical significance")

# ==============================================================
# DATA LOADING
# ==============================================================
print("\n📊 Loading data...")
data = pd.read_csv(file_path)
print(f"📋 Dataset Description:")
print(f"   • Shape: {data.shape}")
print(f"   • Columns: {list(data.columns)}")
print(f"   • Date range: {data['date'].min()} to {data['date'].max()}")
print(f"   • Missing values: {data.isnull().sum().sum()}")
print(f"   • Description: S&P 500 futures data with DIX, GEX, VIX indicators for algorithmic trading")

data.set_index('date', inplace=True)
print(f"✅ Data loaded and indexed by date")

def split_data_temporal(data, train_ratio=0.6, val_ratio=0.2, test_ratio=0.2):
    n = len(data)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))
    
    train_data = data.iloc[:train_end].copy()
    val_data = data.iloc[train_end:val_end].copy()
    test_data = data.iloc[val_end:].copy()
    
    print(f"📈 Temporal Split (No Data Leakage):")
    print(f"   Train: {len(train_data)} samples ({train_data.index[0]} to {train_data.index[-1]})")
    print(f"   Val: {len(val_data)} samples ({val_data.index[0]} to {val_data.index[-1]})")
    print(f"   Test: {len(test_data)} samples ({test_data.index[0]} to {test_data.index[-1]})")
    
    return train_data, val_data, test_data

# ==============================================================
# TRADING ENVIRONMENT - LONG ONLY - CORREGIDO CON RESET COMPLETO
# ==============================================================
class SimpleTradingEnv(gym.Env):
    def __init__(self, indicator_data, price_data):
        super(SimpleTradingEnv, self).__init__()
        
        self.indicator_data = np.array(indicator_data, dtype=np.float32)
        self.price_data = np.array(price_data, dtype=np.float32)
        
        # Ensure same length
        min_len = min(len(self.indicator_data), len(self.price_data))
        self.indicator_data = self.indicator_data[:min_len]
        self.price_data = self.price_data[:min_len]
        
        # Initialize state variables
        self.initial_balance = 100000
        self.reset()
        
        # Observation space
        if len(self.indicator_data.shape) == 1:
            obs_shape = (1,)
        else:
            obs_shape = (self.indicator_data.shape[1],)
            
        self.observation_space = gym.spaces.Box(
            low=-10.0, high=10.0, shape=obs_shape, dtype=np.float32
        )
        # ONLY 2 ACTIONS: 0: Hold/Sell, 1: Buy
        self.action_space = gym.spaces.Discrete(2)

    def reset(self):
        # RESET COMPLETO de todas las variables
        self.current_step = 0
        self.balance = float(self.initial_balance)  # Ensure float
        self.position = 0
        self.entry_price = 0.0  # Ensure float
        self.balance_history = [float(self.initial_balance)]
        self.trades = 0
        
        # IMPORTANTE: Resetear también el estado interno
        self._last_action = None
        self._episode_steps = 0
        
        if len(self.indicator_data.shape) == 1:
            return np.array([self.indicator_data[0]], dtype=np.float32)
        else:
            return self.indicator_data[0].astype(np.float32)

    def step(self, action):
        if self.current_step >= len(self.price_data) - 1:
            # Close any open position at the end
            if self.position == 1:
                current_price = self.price_data[self.current_step]
                # Sell position (incluye spread y comisiones)
                pnl = (current_price - self.entry_price - SPREAD) * POINT_VALUE - COMMISSION_PER_TRADE
                self.balance += pnl
                self.position = 0
                self.trades += 1
            
            self.balance_history.append(self.balance)
            obs = np.zeros(self.observation_space.shape, dtype=np.float32)
            return obs, 0, True, {"balance": self.balance, "trades": self.trades}
        
        current_price = self.price_data[self.current_step]
        reward = 0
        
        # LONG ONLY ACTION LOGIC
        if action == 1:  # BUY
            if self.position == 0:  # No position -> Open long
                self.position = 1
                self.entry_price = current_price + SPREAD  # Include spread
                self.balance -= COMMISSION_PER_TRADE
                reward -= COMMISSION_PER_TRADE / 1000
                self.trades += 1
                
        elif action == 0:  # HOLD/SELL
            if self.position == 1:  # Long position -> Close long
                pnl = (current_price - self.entry_price - SPREAD) * POINT_VALUE - COMMISSION_PER_TRADE
                self.balance += pnl
                reward += pnl / 1000
                self.position = 0
                self.trades += 1
        
        # Calculate current balance (with unrealized PnL if in position)
        current_balance = self.balance
        if self.position == 1:
            unrealized_pnl = (current_price - self.entry_price - SPREAD) * POINT_VALUE
            current_balance += unrealized_pnl
        
        self.balance_history.append(current_balance)
        
        self.current_step += 1
        done = self.current_step >= len(self.price_data) - 1
        
        # Next observation
        if not done:
            if len(self.indicator_data.shape) == 1:
                obs = np.array([self.indicator_data[self.current_step]], dtype=np.float32)
            else:
                obs = self.indicator_data[self.current_step].astype(np.float32)
        else:
            obs = np.zeros(self.observation_space.shape, dtype=np.float32)
        
        info = {"balance": current_balance, "trades": self.trades, "position": self.position}
        return obs, reward, done, info

    def get_balance_history(self):
        return self.balance_history.copy()

# ==============================================================
# 🧠 ADVANCED FEATURE EXTRACTOR
# ==============================================================
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = th.arange(max_len).unsqueeze(1)
        div_term = th.exp(th.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe = th.zeros(max_len, d_model)
        pe[:, 0::2] = th.sin(position * div_term)
        if d_model % 2 == 1:
            pe[:, 1::2] = th.cos(position * div_term[:-1])
        else:
            pe[:, 1::2] = th.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class AdvancedFeatureExtractor(nn.Module):
    def __init__(self, observation_space, features_dim=64, n_heads=4, dropout=0.1):
        super(AdvancedFeatureExtractor, self).__init__()
        self.features_dim = features_dim
        input_dim = observation_space.shape[0]
        
        print(f"🧠 Creating AdvancedFeatureExtractor with input_dim={input_dim}, features_dim={features_dim}")
        
        hidden_dim = min(128, max(64, input_dim * 16))
        
        self.ff_encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout)
        )
        
        attention_dim = hidden_dim // 2
        self.attention = MultiheadAttention(
            embed_dim=attention_dim, 
            num_heads=min(n_heads, attention_dim // 16),
            dropout=dropout, 
            batch_first=True
        )
        self.attention_norm = nn.LayerNorm(attention_dim)
        
        self.bilstm = nn.LSTM(
            attention_dim, 
            attention_dim // 2, 
            bidirectional=True, 
            batch_first=True,
            dropout=dropout if dropout > 0 else 0
        )
        self.lstm_norm = nn.LayerNorm(attention_dim)
        
        self.projection = nn.Sequential(
            nn.Linear(attention_dim, features_dim * 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(features_dim * 2, features_dim),
            nn.Tanh()
        )
        
        self.positional_encoding = PositionalEncoding(attention_dim, dropout=dropout)
        
        print(f"✅ AdvancedFeatureExtractor created successfully!")
        
    def forward(self, observations):
        original_shape = observations.shape
        if len(original_shape) == 1:
            observations = observations.unsqueeze(0)
        
        x = self.ff_encoder(observations)
        x = x.unsqueeze(1)
        x = self.positional_encoding(x)
        
        x_attn, _ = self.attention(x, x, x)
        x = self.attention_norm(x + x_attn)
        
        lstm_out, _ = self.bilstm(x)
        x = self.lstm_norm(lstm_out)
        
        x = x.squeeze(1)
        features = self.projection(x)
        
        if len(original_shape) == 1:
            features = features.squeeze(0)
            
        return features

# ==============================================================
# 📊 COMPLETE METRICS CALCULATION
# ==============================================================
def calculate_complete_metrics(balance_history, price_data, benchmark_returns=None):
    """Calculate ALL requested metrics including Alpha, Beta, Sortino, etc."""
    
    if len(balance_history) < 2:
        return {
            "Alpha": 0.0, "Beta": 0.0, "Sharpe_Ratio": 0.0, "Sortino_Ratio": 0.0,
            "Max_Drawdown": 0.0, "Win_Rate": 0.0, "Max_Loss": 0.0, "RoMaD": 0.0,
            "CAGR": 0.0, "Volatility": 0.0, "T_Statistic": 0.0, "P_Value": 1.0,
            "Total_Return": 0.0
        }
    
    balance_history = np.array(balance_history)
    returns = np.diff(balance_history) / balance_history[:-1]
    returns = returns[np.isfinite(returns)]
    
    if len(returns) < 2:
        return {
            "Alpha": 0.0, "Beta": 0.0, "Sharpe_Ratio": 0.0, "Sortino_Ratio": 0.0,
            "Max_Drawdown": 0.0, "Win_Rate": 0.0, "Max_Loss": 0.0, "RoMaD": 0.0,
            "CAGR": 0.0, "Volatility": 0.0, "T_Statistic": 0.0, "P_Value": 1.0,
            "Total_Return": 0.0
        }
    
    # Basic metrics
    total_return = (balance_history[-1] - balance_history[0]) / balance_history[0]
    
    # CAGR
    n_years = len(balance_history) / 252
    if n_years > 0 and balance_history[0] > 0:
        cagr = (balance_history[-1] / balance_history[0]) ** (1 / n_years) - 1
    else:
        cagr = 0
    
    # Volatility (Annualized)
    volatility = np.std(returns) * np.sqrt(252) if len(returns) > 1 else 0
    
    # Sharpe Ratio
    risk_free_rate = 0.02
    excess_returns = returns - (risk_free_rate / 252)
    sharpe_ratio = np.mean(excess_returns) / np.std(returns) * np.sqrt(252) if np.std(returns) > 0 else 0
    
    # Drawdown metrics
    peak = np.maximum.accumulate(balance_history)
    drawdown = (peak - balance_history) / np.maximum(peak, 1)
    max_drawdown = np.max(drawdown)
    
    # Sortino Ratio
    negative_returns = returns[returns < 0]
    downside_std = np.std(negative_returns) * np.sqrt(252) if len(negative_returns) > 1 else 0.001
    sortino_ratio = np.mean(excess_returns) * np.sqrt(252) / downside_std if downside_std > 0 else 0
    
    # Win Rate
    positive_returns = returns[returns > 0]
    win_rate = len(positive_returns) / len(returns) if len(returns) > 0 else 0
    
    # Max Loss
    max_loss = np.min(returns) if len(returns) > 0 else 0
    
    # RoMaD
    romad = cagr / max_drawdown if max_drawdown > 0 else 0
    
    # T-statistic and P-value
    if len(returns) > 1:
        t_stat, p_value = stats.ttest_1samp(returns, 0)
        t_stat = float(t_stat) if not np.isnan(t_stat) else 0
        p_value = float(p_value) if not np.isnan(p_value) else 1
    else:
        t_stat, p_value = 0, 1
    
    # Alpha and Beta
    if benchmark_returns is not None and len(benchmark_returns) == len(returns):
        try:
            covariance = np.cov(returns, benchmark_returns)[0][1]
            benchmark_variance = np.var(benchmark_returns)
            beta = covariance / benchmark_variance if benchmark_variance > 0 else 0
            
            benchmark_mean = np.mean(benchmark_returns)
            returns_mean = np.mean(returns)
            alpha = returns_mean - (risk_free_rate/252 + beta * (benchmark_mean - risk_free_rate/252))
            alpha = alpha * 252  # Annualize
            
        except:
            alpha, beta = 0, 0
    else:
        market_return = 0.08 / 252
        market_returns = np.full(len(returns), market_return)
        
        try:
            covariance = np.cov(returns, market_returns)[0][1]
            market_variance = np.var(market_returns)
            beta = covariance / market_variance if market_variance > 0 else 0
            
            returns_mean = np.mean(returns)
            alpha = returns_mean - (risk_free_rate/252 + beta * (market_return - risk_free_rate/252))
            alpha = alpha * 252
        except:
            alpha, beta = 0, 0
    
    return {
        "Alpha": float(alpha),
        "Beta": float(beta),
        "Sharpe_Ratio": float(sharpe_ratio),
        "Sortino_Ratio": float(sortino_ratio),
        "Max_Drawdown": float(max_drawdown),
        "Win_Rate": float(win_rate),
        "Max_Loss": float(max_loss),
        "RoMaD": float(romad),
        "CAGR": float(cagr),
        "Volatility": float(volatility),
        "T_Statistic": float(t_stat),
        "P_Value": float(p_value),
        "Total_Return": float(total_return)
    }

# ==============================================================
# 🔬 DENOISING ANALYSIS
# ==============================================================
def analyze_denoising_effectiveness(original_data, denoised_data, wavelet_name, save_path):
    """Analyze and visualize denoising effectiveness"""
    
    plt.figure(figsize=(15, 10))
    
    # Original vs Denoised
    plt.subplot(2, 3, 1)
    plt.plot(original_data[:200], label='Original', alpha=0.7, linewidth=1)
    plt.plot(denoised_data[:200], label='Denoised', alpha=0.8, linewidth=1.5)
    plt.title(f'Signal Comparison - {wavelet_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Noise removed
    plt.subplot(2, 3, 2)
    noise = original_data - denoised_data
    plt.plot(noise[:200], color='red', alpha=0.7)
    plt.title('Removed Noise')
    plt.grid(True, alpha=0.3)
    
    # FFT Analysis
    plt.subplot(2, 3, 3)
    freqs_orig = np.fft.fftfreq(len(original_data))
    fft_orig = np.abs(np.fft.fft(original_data))
    fft_clean = np.abs(np.fft.fft(denoised_data))
    plt.semilogy(freqs_orig[:len(freqs_orig)//2], fft_orig[:len(fft_orig)//2], alpha=0.7, label='Original')
    plt.semilogy(freqs_orig[:len(freqs_orig)//2], fft_clean[:len(fft_clean)//2], alpha=0.8, label='Denoised')
    plt.title('Frequency Domain Analysis')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Wavelet decomposition visualization
    plt.subplot(2, 3, 4)
    coeffs = pywt.wavedec(original_data, wavelet_name, level=2)
    plt.plot(coeffs[0][:100], label='Approximation', linewidth=1.5)
    plt.title('Approximation Coefficients (L2)')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 3, 5)
    plt.plot(coeffs[1][:50], label='Detail L1', alpha=0.8)
    plt.plot(coeffs[2][:50], label='Detail L2', alpha=0.8)
    plt.title('Detail Coefficients')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Metrics calculation
    signal_power = np.var(denoised_data)
    noise_power = np.var(noise)
    snr = 10 * np.log10(signal_power / noise_power) if noise_power > 0 else float('inf')
    noise_reduction = (np.std(noise)/np.std(original_data)*100)
    
    plt.subplot(2, 3, 6)
    plt.text(0.1, 0.8, f'SNR Improvement: {snr:.2f} dB', fontsize=12, transform=plt.gca().transAxes)
    plt.text(0.1, 0.7, f'Noise Reduction: {noise_reduction:.1f}%', fontsize=12, transform=plt.gca().transAxes)
    plt.text(0.1, 0.6, f'Wavelet: {wavelet_name}', fontsize=12, transform=plt.gca().transAxes)
    plt.text(0.1, 0.5, f'Decomposition Level: 2', fontsize=12, transform=plt.gca().transAxes)
    plt.text(0.1, 0.4, f'Threshold: Conservative (0.1 * σ)', fontsize=10, transform=plt.gca().transAxes)
    plt.text(0.1, 0.3, f'Method: Soft thresholding', fontsize=10, transform=plt.gca().transAxes)
    plt.text(0.1, 0.2, f'Justification: Level 2 provides good', fontsize=10, transform=plt.gca().transAxes)
    plt.text(0.1, 0.1, f'balance between denoising and signal preservation', fontsize=10, transform=plt.gca().transAxes)
    plt.axis('off')
    plt.title('Denoising Metrics')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return snr, noise_power, noise_reduction

# ==============================================================
# 📊 STATISTICAL SIGNIFICANCE ANALYSIS
# ==============================================================
def statistical_significance_analysis(results_df, save_path):
    """Perform statistical significance tests"""
    
    algorithms = results_df['Algorithm'].unique()
    metrics = ['Sharpe_Ratio', 'Total_Return', 'Max_Drawdown', 'Alpha', 'Sortino_Ratio']
    
    significance_results = {}
    
    for metric in metrics:
        significance_results[metric] = {}
        for i, algo1 in enumerate(algorithms):
            for algo2 in algorithms[i+1:]:
                data1 = results_df[results_df['Algorithm'] == algo1][metric]
                data2 = results_df[results_df['Algorithm'] == algo2][metric]
                
                if len(data1) > 1 and len(data2) > 1:
                    t_stat, p_value = stats.ttest_ind(data1, data2)
                    significance_results[metric][f'{algo1}_vs_{algo2}'] = {
                        't_statistic': t_stat,
                        'p_value': p_value,
                        'significant': p_value < 0.05,
                        'mean_diff': float(data1.mean() - data2.mean())
                    }
    
    # Create significance table and save
    with open(save_path, 'w') as f:
        f.write("STATISTICAL SIGNIFICANCE ANALYSIS\n")
        f.write("="*60 + "\n\n")
        
        for metric in metrics:
            f.write(f"\n{metric.upper()}:\n")
            f.write("-" * 40 + "\n")
            for comparison, stats_result in significance_results[metric].items():
                significance = "SIGNIFICANT" if stats_result['significant'] else "NOT SIGNIFICANT"
                f.write(f"{comparison}: p={stats_result['p_value']:.4f} ({significance})\n")
                f.write(f"  Mean difference: {stats_result['mean_diff']:.4f}\n")
    
    print(f"✅ Statistical significance analysis saved to: {save_path}")
    return significance_results

# ==============================================================
# 🤖 CLASSICAL METHODS IMPLEMENTATION
# ==============================================================
def implement_classical_methods(train_features, train_labels, test_features, test_prices):
    """Implement classical ML methods including XGBoost"""
    
    classical_results = {}
    
    # 1. XGBoost
    print("  🚀 Training XGBoost...")
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        eval_metric='logloss'
    )
    xgb_model.fit(train_features, train_labels)
    xgb_predictions = xgb_model.predict(test_features)
    xgb_balance = simulate_trading_strategy(xgb_predictions, test_prices)
    classical_results['XGBoost'] = calculate_complete_metrics(xgb_balance, test_prices)
    
    # 2. Random Forest
    print("  🌲 Training Random Forest...")
    rf_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42
    )
    rf_model.fit(train_features, train_labels)
    rf_predictions = rf_model.predict(test_features)
    rf_balance = simulate_trading_strategy(rf_predictions, test_prices)
    classical_results['Random_Forest'] = calculate_complete_metrics(rf_balance, test_prices)
    
    # 3. Logistic Regression
    print("  📊 Training Logistic Regression...")
    lr_model = LogisticRegression(random_state=42)
    lr_model.fit(train_features, train_labels)
    lr_predictions = lr_model.predict(test_features)
    lr_balance = simulate_trading_strategy(lr_predictions, test_prices)
    classical_results['Logistic_Regression'] = calculate_complete_metrics(lr_balance, test_prices)
    
    # 4. Buy and Hold
    print("  💰 Calculating Buy & Hold...")
    buy_hold_return = (test_prices[-1] - test_prices[0]) / test_prices[0]
    buy_hold_balance = [100000 * (1 + (price - test_prices[0]) / test_prices[0]) for price in test_prices]
    classical_results['Buy_and_Hold'] = calculate_complete_metrics(buy_hold_balance, test_prices)
    
    # 5. Moving Average Crossover
    print("  📈 Calculating Moving Average Strategy...")
    ma_short = pd.Series(test_prices).rolling(20).mean()
    ma_long = pd.Series(test_prices).rolling(50).mean()
    ma_signals = np.where(ma_short > ma_long, 1, 0)
    ma_balance = simulate_trading_strategy(ma_signals, test_prices)
    classical_results['Moving_Average'] = calculate_complete_metrics(ma_balance, test_prices)
    
    return classical_results

def create_trading_labels(price_data, lookforward=5):
    """Create trading labels for supervised learning"""
    labels = []
    for i in range(len(price_data) - lookforward):
        future_return = (price_data[i + lookforward] - price_data[i]) / price_data[i]
        labels.append(1 if future_return > 0.002 else 0)  # 0.2% threshold
    
    return np.array(labels)

def simulate_trading_strategy(signals, prices):
    """Simulate trading strategy given signals and prices"""
    balance = 100000
    position = 0
    balance_history = [balance]
    
    for i, signal in enumerate(signals):
        current_price = prices[i]
        
        if signal == 1 and position == 0:  # Buy signal
            position = 1
            entry_price = current_price + SPREAD
            balance -= COMMISSION_PER_TRADE
        elif signal == 0 and position == 1:  # Sell signal
            pnl = (current_price - entry_price - SPREAD) * POINT_VALUE - COMMISSION_PER_TRADE
            balance += pnl
            position = 0
        
        # Calculate current balance
        current_balance = balance
        if position == 1:
            unrealized_pnl = (current_price - entry_price - SPREAD) * POINT_VALUE
            current_balance += unrealized_pnl
        
        balance_history.append(current_balance)
    
    return balance_history

# ==============================================================
# WAVELET PROCESSING
# ==============================================================
def simple_wavelet_denoise(series, wavelet):
    """Simplified wavelet denoising with analysis"""
    try:
        coeffs = pywt.wavedec(series, wavelet, level=2)
        threshold = np.std(coeffs[-1]) * 0.1
        coeffs[1:] = [pywt.threshold(c, threshold, 'soft') for c in coeffs[1:]]
        return pywt.waverec(coeffs, wavelet)[:len(series)]
    except:
        return series

# ==============================================================
# HYPERPARAMETER TUNING - CORREGIDO CON MÁS COMBINACIONES
# ==============================================================
def realistic_hyperparameter_tuning(algo_class, train_features, val_features, train_prices, val_prices):
    """Hyperparameter tuning with realistic timesteps"""
    
    param_grid = HYPERPARAM_GRID[algo_class.__name__]
    best_model = None
    best_score = -np.inf
    best_params = {}
    
    # CORREGIDO: Generar más combinaciones para A2C y DQN
    if algo_class.__name__ == 'PPO':
        combinations = [
            {'learning_rate': param_grid['learning_rate'][0], 'gamma': param_grid['gamma'][0], 
             'n_steps': param_grid['n_steps'][0], 'batch_size': param_grid['batch_size'][0]},
            {'learning_rate': param_grid['learning_rate'][1], 'gamma': param_grid['gamma'][1], 
             'n_steps': param_grid['n_steps'][0], 'batch_size': param_grid['batch_size'][0]}
        ]
    elif algo_class.__name__ == 'A2C':
        combinations = [
            {'learning_rate': param_grid['learning_rate'][0], 'gamma': param_grid['gamma'][0], 
             'n_steps': param_grid['n_steps'][0]},
            {'learning_rate': param_grid['learning_rate'][1], 'gamma': param_grid['gamma'][1], 
             'n_steps': param_grid['n_steps'][0]},
            {'learning_rate': param_grid['learning_rate'][2], 'gamma': param_grid['gamma'][0], 
             'n_steps': param_grid['n_steps'][1]},  # Nueva combinación
        ]
    else:  # DQN
        combinations = [
            {'learning_rate': param_grid['learning_rate'][0], 'gamma': param_grid['gamma'][0], 
             'buffer_size': param_grid['buffer_size'][0], 'exploration_fraction': param_grid['exploration_fraction'][0]},
            {'learning_rate': param_grid['learning_rate'][1], 'gamma': param_grid['gamma'][1], 
             'buffer_size': param_grid['buffer_size'][0], 'exploration_fraction': param_grid['exploration_fraction'][0]},
            {'learning_rate': param_grid['learning_rate'][2], 'gamma': param_grid['gamma'][0], 
             'buffer_size': param_grid['buffer_size'][1], 'exploration_fraction': param_grid['exploration_fraction'][1]},  # Nueva combinación
        ]
    
    print(f"🔍 Testing {len(combinations)} combinations with {TRAINING_TIMESTEPS['hyperparameter_tuning']:,} timesteps")
    
    for i, params in enumerate(combinations):
        print(f"      ⚙️ {i+1}/{len(combinations)}: {params}")
        
        try:
            train_env = DummyVecEnv([lambda: SimpleTradingEnv(train_features.copy(), train_prices.copy())])
            train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True)
            
            policy_kwargs = dict(
                features_extractor_class=AdvancedFeatureExtractor,
                features_extractor_kwargs=dict(features_dim=64, n_heads=4, dropout=0.1)
            )
            
            model = algo_class('MlpPolicy', train_env, verbose=0, policy_kwargs=policy_kwargs, **params)
            model.learn(total_timesteps=TRAINING_TIMESTEPS['hyperparameter_tuning'])
            
            # Validation
            val_env = DummyVecEnv([lambda: SimpleTradingEnv(val_features.copy(), val_prices.copy())])
            val_env = VecNormalize(val_env, norm_obs=True, norm_reward=False, training=False)
            # CORREGIDO: Copiar estadísticas de normalización (no solo referencia)
            val_env.obs_rms = train_env.obs_rms.copy()
            
            obs = val_env.reset()
            val_balance = [100000]
            done = [False]
            
            while not done[0]:
                action, _ = model.predict(obs, deterministic=True)
                obs, _, done, info = val_env.step(action)
                val_balance.append(info[0]['balance'])
            
            val_metrics = calculate_complete_metrics(val_balance, val_prices)
            score = val_metrics['Sharpe_Ratio'] + val_metrics['Sortino_Ratio'] + val_metrics['Total_Return'] - val_metrics['Max_Drawdown']
            
            print(f"        📊 Score: {score:.3f}")
            
            if score > best_score:
                best_score = score
                best_model = model
                best_params = params.copy()
                best_params['train_env'] = train_env
                best_params['score'] = score
            
            val_env.close()
            if best_params.get('train_env') != train_env:
                train_env.close()
                
        except Exception as e:
            print(f"        ❌ Error: {str(e)}")
            continue
    
    return best_model, best_params

# ==============================================================
# MAIN PROCESSING LOOP - CORREGIDO CON SEMILLAS ÚNICAS
# ==============================================================
print("\n🚀 Starting COMPLETE ANALYSIS with ALL IMPROVEMENTS...")

# Split data
train_data, val_data, test_data = split_data_temporal(data)
benchmark_returns = np.diff(test_data['PRICE'].values) / test_data['PRICE'].values[:-1]

# Initialize results
all_rl_results = []
all_classical_results = []
denoising_analysis_results = []

total_experiments = sum(len(wavelets) for wavelets in wavelet_families.values()) * 3 * len(MAIN_FEATURE_COMBINATIONS)
current_experiment = 0

print(f"\n📊 TOTAL EXPERIMENTS TO RUN: {total_experiments}")
print(f"⏱️ Estimated time: ~{total_experiments * 8 / 60:.1f} hours")

# Process each family
for family_name, wavelets in wavelet_families.items():
    print(f"\n{'='*50}")
    print(f"🌊 PROCESSING: {family_name}")
    print(f"{'='*50}")
    
    for wavelet in wavelets:
        print(f"\n📡 Wavelet: {wavelet}")
        
        try:
            # Process all datasets
            processed_data = {}
            
            for dataset_name, dataset in [('train', train_data), ('val', val_data), ('test', test_data)]:
                price_data = dataset['PRICE'].values
                indicator_data = dataset[['DIX', 'GEX', 'VIX']].values
                
                # Denoising with analysis
                denoised_data = []
                for col_idx, col_name in enumerate(['DIX', 'GEX', 'VIX']):
                    series = indicator_data[:, col_idx]
                    if np.any(np.isnan(series)):
                        series = pd.Series(series).fillna(method='ffill').fillna(0).values
                    
                    denoised = simple_wavelet_denoise(series, wavelet)
                    denoised_data.append(denoised)
                    
                    # Analyze denoising effectiveness for test data only
                    if dataset_name == 'test':
                        denoising_save_path = os.path.join(results_dir, 'denoising_analysis', 
                                                         f'denoising_{wavelet}_{col_name}.png')
                        snr, noise_power, noise_reduction = analyze_denoising_effectiveness(
                            series, denoised, wavelet, denoising_save_path)
                        
                        denoising_analysis_results.append({
                            'Wavelet': wavelet,
                            'Indicator': col_name,
                            'SNR_Improvement': snr,
                            'Noise_Reduction_Percent': noise_reduction,
                            'Noise_Power': noise_power
                        })
                
                denoised_data = np.column_stack(denoised_data)
                
                # Normalization
                if dataset_name == 'train':
                    scaler = MinMaxScaler()
                    normalized_data = scaler.fit_transform(denoised_data)
                    processed_data['scaler'] = scaler
                else:
                    normalized_data = processed_data['scaler'].transform(denoised_data)
                
                processed_data[dataset_name] = {
                    'price': price_data,
                    'indicators': normalized_data
                }
            
            # Feature mapping
            feature_mapping = {
                'DIX': [0], 'GEX': [1], 'VIX': [2],
                'DIX_GEX': [0, 1], 'DIX_VIX': [0, 2], 'GEX_VIX': [1, 2],
                'DIX_GEX_VIX': [0, 1, 2]
            }
            
            # Process RL algorithms
            algorithms = {'PPO': PPO, 'A2C': A2C, 'DQN': DQN}
            
            for algo_name, algo_class in algorithms.items():
                print(f"\n  🤖 {algo_name}")
                
                for feature_comb in MAIN_FEATURE_COMBINATIONS:
                    current_experiment += 1
                    
                    if len(feature_comb) == 1:
                        feature_name = feature_comb[0]
                    else:
                        feature_name = '_'.join(feature_comb)
                    
                    feature_indices = feature_mapping[feature_name]
                    
                    # CORREGIDO: Semilla única por algoritmo, wavelet y features
                    algo_seed = hash(f"{algo_name}_{wavelet}_{feature_name}_{current_experiment}") % (2**32)
                    set_random_seeds(algo_seed)
                    
                    print(f"    📊 [{current_experiment}/{total_experiments}] {feature_name} (seed: {algo_seed})")
                    
                    try:
                        # Extract features
                        train_features = processed_data['train']['indicators'][:, feature_indices]
                        val_features = processed_data['val']['indicators'][:, feature_indices]
                        test_features = processed_data['test']['indicators'][:, feature_indices]
                        
                        # Hyperparameter tuning
                        best_model, best_params = realistic_hyperparameter_tuning(
                            algo_class, train_features, val_features,
                            processed_data['train']['price'], processed_data['val']['price']
                        )
                        
                        if best_model is None:
                            print(f"      ❌ No valid model")
                            continue
                        
                        # Final training
                        print(f"      🎯 Final training with {TRAINING_TIMESTEPS['final_training']:,} timesteps...")
                        train_env = DummyVecEnv([lambda: SimpleTradingEnv(train_features.copy(), processed_data['train']['price'].copy())])
                        train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True)
                        
                        policy_kwargs = dict(
                            features_extractor_class=AdvancedFeatureExtractor,
                            features_extractor_kwargs=dict(features_dim=64, n_heads=4, dropout=0.1)
                        )
                        
                        final_params = {k: v for k, v in best_params.items() if k not in ['train_env', 'score']}
                        model = algo_class('MlpPolicy', train_env, verbose=0, policy_kwargs=policy_kwargs, **final_params)
                        model.learn(total_timesteps=TRAINING_TIMESTEPS['final_training'])
                        
                        # Test evaluation
                        print(f"      📊 Testing...")
                        test_env = SimpleTradingEnv(test_features.copy(), processed_data['test']['price'].copy())
                        
                        obs = test_env.reset()
                        done = False
                        step_count = 0
                        
                        while not done and step_count < len(test_features):
                            if hasattr(train_env, 'obs_rms'):
                                normalized_obs = (obs - train_env.obs_rms.mean) / np.sqrt(train_env.obs_rms.var + 1e-8)
                            else:
                                normalized_obs = obs
                            
                            action, _ = model.predict(normalized_obs.reshape(1, -1), deterministic=True)
                            obs, reward, done, info = test_env.step(action[0] if hasattr(action, '__getitem__') else action)
                            step_count += 1
                        
                        # Results
                        test_balance = test_env.get_balance_history()
                        test_metrics = calculate_complete_metrics(test_balance, processed_data['test']['price'], benchmark_returns)
                        
                        # Store results
                        result = {
                            'Method_Type': 'Reinforcement_Learning',
                            'Wavelet_Family': family_name,
                            'Wavelet': wavelet,
                            'Algorithm': algo_name,
                            'Features': feature_name,
                            'Feature_Count': len(feature_indices),
                            'Feature_Extractor': 'Advanced',
                            'Strategy': 'Long-Only',
                            'Training_Timesteps': TRAINING_TIMESTEPS['final_training'],
                            'Random_Seed': algo_seed,  # AÑADIDO
                            **test_metrics,
                            'Best_Learning_Rate': best_params.get('learning_rate', 0),
                            'Best_Gamma': best_params.get('gamma', 0),
                            'Validation_Score': best_params.get('score', 0)
                        }
                        
                        all_rl_results.append(result)
                        
                        print(f"      ✅ Sharpe: {test_metrics['Sharpe_Ratio']:.3f}, Return: {test_metrics['Total_Return']*100:.1f}%")
                        
                        # Cleanup
                        train_env.close()
                        
                    except Exception as e:
                        print(f"      ❌ Error: {str(e)}")
                        continue
            
            # Classical methods (once per wavelet, using best feature combination)
            print(f"\n  🏛️ Classical Methods for {wavelet}")
            try:
                best_features = processed_data['train']['indicators']  # Use all features
                test_features_classical = processed_data['test']['indicators']
                
                # Create labels for supervised learning
                train_labels = create_trading_labels(processed_data['train']['price'])
                # Ensure features and labels have same length
                min_len = min(len(best_features), len(train_labels))
                best_features = best_features[:min_len]
                train_labels = train_labels[:min_len]
                
                classical_results = implement_classical_methods(
                    best_features, train_labels, test_features_classical, processed_data['test']['price']
                )
                
                for method_name, metrics in classical_results.items():
                    classical_result = {
                        'Method_Type': 'Classical',
                        'Wavelet_Family': family_name,
                        'Wavelet': wavelet,
                        'Algorithm': method_name,
                        'Features': 'DIX_GEX_VIX',
                        'Feature_Count': 3,
                        'Feature_Extractor': 'None',
                        'Strategy': 'Long-Only',
                        'Training_Timesteps': 0,
                        'Random_Seed': 0,  # AÑADIDO
                        **metrics,
                        'Best_Learning_Rate': 0,
                        'Best_Gamma': 0,
                        'Validation_Score': 0
                    }
                    all_classical_results.append(classical_result)
                
                print(f"      ✅ Classical methods completed")
                
            except Exception as e:
                print(f"      ❌ Classical methods error: {str(e)}")
                continue
                        
        except Exception as e:
            print(f"❌ Error with {wavelet}: {str(e)}")
            continue

# ==============================================================
# SAVE COMPLETE RESULTS
# ==============================================================
print(f"\n{'='*60}")
print(f"💾 SAVING COMPLETE RESULTS")
print(f"{'='*60}")

if all_rl_results or all_classical_results:
    # Combine all results
    all_results = all_rl_results + all_classical_results
    results_df = pd.DataFrame(all_results)
    
    # Save main results
    main_csv = os.path.join(results_dir, 'out_of_sample', 'complete_results_with_classical_methods_FIXED.csv')
    results_df.to_csv(main_csv, index=False)
    print(f"✅ Complete Results saved: {main_csv}")
    
    # Save denoising analysis
    if denoising_analysis_results:
        denoising_df = pd.DataFrame(denoising_analysis_results)
        denoising_csv = os.path.join(results_dir, 'denoising_analysis', 'denoising_effectiveness.csv')
        denoising_df.to_csv(denoising_csv, index=False)
        print(f"✅ Denoising analysis saved: {denoising_csv}")
    
    # Statistical significance analysis
    significance_path = os.path.join(results_dir, 'academic_reports', 'statistical_significance.txt')
    statistical_significance_analysis(results_df, significance_path)
    
    # VERIFICAR DIFERENCIAS ENTRE ALGORITMOS
    print(f"\n🔍 VERIFICANDO RESULTADOS:")
    rl_results = results_df[results_df['Method_Type'] == 'Reinforcement_Learning']
    
    for algo in ['A2C', 'DQN', 'PPO']:
        algo_data = rl_results[rl_results['Algorithm'] == algo]
        if len(algo_data) > 0:
            best_result = algo_data.loc[algo_data['Sharpe_Ratio'].idxmax()]
            print(f"  {algo}: Sharpe={best_result['Sharpe_Ratio']:.4f}, "
                  f"Seed={best_result['Random_Seed']}, "
                  f"Wavelet={best_result['Wavelet']}, "
                  f"Features={best_result['Features']}")
    
    print(f"\n✅ CORRECCIONES APLICADAS:")
    print(f"  ✓ Semillas aleatorias únicas por experimento")
    print(f"  ✓ Reset completo del environment")
    print(f"  ✓ Más variación en hiperparámetros A2C/DQN")
    print(f"  ✓ Normalización mejorada")
    print(f"  ✓ Registro de semillas en resultados")
    
else:
    print("❌ No results to save. Check for errors in the processing loop.")

print(f"\n{'='*60}")
print(f"✅ COMPLETE ANALYSIS FINISHED!")
print(f"📁 All results saved in: {results_dir}")
print(f"{'='*60}")